In [1]:
import sys, os; sys.path.append(os.path.abspath(".."))
from src.data_prep import load_data
from src.features import build_features

df = load_data(save_processed=False)
df_fe = build_features(df)  

# Split into features (X) and target (y) for modeling
y = df_fe["home_price_index"]
X = df_fe.drop(columns=["home_price_index"])
df_fe.head()

No missing values found.


,building_permits,construction_price_index,delinquency_rate,GDP,housing_inventory,housing_subsidies,income,interest_rate,mortgage_rate,construction_unit,...,mortgage_rate_lag1,mortgage_rate_lag3,interest_rate_lag1,interest_rate_lag3,unemployment_rate_lag1,unemployment_rate_lag3,housing_inventory_lag1,housing_inventory_lag3,GDP_lag1,GDP_lag3
DATE,,,,,,,,,,,,,,,,,,,,,
2003-04-01,1803,145.9,1.83,14743.567,91,25.93,10752.7,1.26,5.8125,1662,...,5.7450,5.9160,1.25,1.24,5.9,5.8,98.0,76.0,14614.141,14614.141
2003-05-01,1835,145.8,1.83,14743.567,101,25.93,10832.0,1.26,5.4840,1733,...,5.8125,5.8425,1.26,1.26,6.0,5.9,91.0,82.0,14743.567,14614.141
2003-06-01,1875,146.1,1.83,14743.567,107,25.93,10860.6,1.22,5.2300,1641,...,5.4840,5.7450,1.26,1.25,6.1,5.9,101.0,98.0,14743.567,14614.141
2003-07-01,1885,147.0,1.73,14988.782,99,25.93,10991.1,1.01,5.6325,1680,...,5.2300,5.8125,1.22,1.26,6.3,6.0,107.0,91.0,14743.567,14743.567
2003-08-01,1966,147.2,1.73,14988.782,105,25.93,11066.7,1.03,6.2640,1570,...,5.6325,5.4840,1.01,1.26,6.2,6.1,99.0,101.0,14988.782,14743.567
